In [21]:
import wget
from osgeo import gdal
import rasterstats as rs
import geopandas as gpd
import pandas as pd
import os
import glob
import sys

## Download Daily NARR Precipitation Data (apcp)

#### Find Year Range of our Strips

In [2]:
index = pd.read_pickle('./Grids/data/index.pkl')

In [3]:
index['acquisitio'].min()

'2008-03-30'

In [4]:
index['acquisitio'].max()

'2017-09-03'

In [25]:
out_dir = '../data/narr/netcdf/'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
url = 'ftp://ftp.cdc.noaa.gov/Datasets/NARR/Dailies/monolevel/apcp.'

for year in range(2007, 2019):
    print(year)
    download_url = url + str(year) + '.nc'
    outfile = out_dir + 'apcp.' + str(year) + '.nc'
    wget.download(download_url, outfile)

2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


## Convert to GeoTIFF, Project to EPSG 3413

In [27]:
epsg_3413 = '+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs' 
warp_options = gdal.WarpOptions(options = '-t_srs \"' + epsg_3413 + '\" -of GTiff')

out_dir = '../data/narr/geotiff/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

for netcdf in glob.glob('../data/narr/netcdf/*.nc'):
    name = netcdf[netcdf.rfind('/')+1:netcdf.rfind('.')]
    print(name)
    outfile = out_dir + name + '.geotiff'
    ds = gdal.Warp(outfile, 'NETCDF:' + netcdf + ':apcp', options=warp_options)
    ds = None

apcp.2007
apcp.2008
apcp.2009
apcp.2010
apcp.2011
apcp.2012
apcp.2013
apcp.2014
apcp.2015
apcp.2016
apcp.2017
apcp.2018


## Extract Daily Bands

In [39]:
# gdal.Translate(outfile, in_ds, options=translate_options)

for geotiff in glob.glob('../data/narr/geotiff/*.geotiff'):
    src = gdal.Open(geotiff)
    
    name = geotiff[geotiff.rfind('/')+1:geotiff.rfind('.')]
    year = int(name[5:])
    print(year)
    
    out_dir = '../data/narr/daily/' + str(year) + '/'
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        
    num_bands = src.RasterCount
                
    for day in range(1, num_bands+1):
        translate_options = gdal.TranslateOptions(options='-b ' + str(day) + ' -of GTiff')
        outfile = out_dir + str(day) + '.geotiff'
        gdal.Translate(outfile, src, options=translate_options)
        
        


2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


## Zonal Stats on Each Grid

In [5]:
grids = gpd.read_file('../data/grid_shapefile/1km_filtered/filtered.shp')
grids['id'] = grids['id'].astype(int)
grids = grids.set_index('id', drop=True)

In [6]:
grids

,left,top,right,bottom,geometry
id,,,,,
222,-2.138483e+06,624945.32479,-2.137483e+06,623945.32479,"POLYGON ((-2138483.335 624945.325, -2137483.33..."
742,-2.136483e+06,622945.32479,-2.135483e+06,621945.32479,"POLYGON ((-2136483.335 622945.325, -2135483.33..."
1000,-2.135483e+06,623945.32479,-2.134483e+06,622945.32479,"POLYGON ((-2135483.335 623945.325, -2134483.33..."
1750,-2.132483e+06,650945.32479,-2.131483e+06,649945.32479,"POLYGON ((-2132483.335 650945.325, -2131483.33..."
1777,-2.132483e+06,623945.32479,-2.131483e+06,622945.32479,"POLYGON ((-2132483.335 623945.325, -2131483.33..."
...,...,...,...,...,...
42524,-1.974483e+06,798945.32479,-1.973483e+06,797945.32479,"POLYGON ((-1974483.335 798945.325, -1973483.33..."
42782,-1.973483e+06,799945.32479,-1.972483e+06,798945.32479,"POLYGON ((-1973483.335 799945.325, -1972483.33..."
42783,-1.973483e+06,798945.32479,-1.972483e+06,797945.32479,"POLYGON ((-1973483.335 798945.325, -1972483.33..."


In [63]:
precip = pd.DataFrame(columns=pd.date_range(start='01/01/2007', end='12/31/2018'), index=grids.index)

In [ ]:
for year in range(2007, 2019):
    for day_geotiff in glob.glob('../data/narr/daily/' + str(year) + '/*.geotiff'):
        # Format the day into datetime
        day = day_geotiff[day_geotiff.rfind('/')+1:day_geotiff.rfind('.')]
        day = pd.to_datetime(day, format='%j')
        day = day.replace(year=year).date()

        # Extract the Data
        data = rs.zonal_stats(grids, day_geotiff, stats=stats, all_touched=True)
        data = pd.DataFrame.from_dict(data).set_index(grids.index).rename({'mean':day}, axis='columns')
        precip[day] = data